In [184]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from scipy.spatial.distance import mahalanobis

In [185]:
df = pd.read_csv('../data/allinone.csv')

### Manejo de Valores Nulos

In [186]:
# Comprobamos la cantidad de nulos en las columnas
df.isnull().sum()

Unnamed: 0                 0
 Destination Port          0
 Flow Duration             0
 Total Fwd Packets         0
 Total Backward Packets    0
                          ..
Idle Mean                  0
 Idle Std                  0
 Idle Max                  0
 Idle Min                  0
 Label                     0
Length: 80, dtype: int64

In [187]:
# Eliminamos las columnas con >50% de valores nulos 

umbral = len(df) * 0.5 
df = df.dropna(thresh=umbral, axis=1) 

In [188]:
# Verificar valores infinitos en columnas numéricas
inf_check = np.isinf(df.select_dtypes(include=['int64', 'float64'])).sum()
print("Valores infinitos por columna:\n", inf_check)

Valores infinitos por columna:
 Unnamed: 0                 0
 Destination Port          0
 Flow Duration             0
 Total Fwd Packets         0
 Total Backward Packets    0
                          ..
 Active Min                0
Idle Mean                  0
 Idle Std                  0
 Idle Max                  0
 Idle Min                  0
Length: 79, dtype: int64


In [189]:
df = df.replace([np.inf, -np.inf], np.nan)  # Reemplazar inf con NaN

In [190]:
# Imputación para columnas numéricas (todas las int y float)
col_num = df.select_dtypes(include=['int64', 'float64']).columns
imputer_num = SimpleImputer(strategy='mean')
df[col_num] = imputer_num.fit_transform(df[col_num])

In [191]:
# Imputación para columnas categóricas 
col_cat = df.select_dtypes(include=['object']).columns
imputer_cat = SimpleImputer(strategy='most_frequent')
df[col_cat] = imputer_cat.fit_transform(df[col_cat])

In [192]:
# Eliminar filas con nulos residuales 

df = df.dropna()

###  CORREGIR INCONSISTENCIAS 

In [193]:
# Estandarizar formatos en columnas categóricas (texto en minúsculas) 
for i in col_cat:
    df[i] = df[i].str.lower().str.strip()

In [194]:
# Eliminar duplicados 

df=df.drop_duplicates() 

### MANEJO DE OUTLIERS 

In [195]:
# Método IQR con cuantiles para la detección de outliers (no se detectan outliers)
Q1 = df[col_num].quantile(0.25)
Q3 = df[col_num].quantile(0.75)
IQR = Q3 - Q1

# Filtramos las filas cuyos valores estén dentro de los límites
df_filtrado = df[(df[col_num] >= (Q1 - 1.5 * IQR)) & (df[col_num] <= (Q3 + 1.5 * IQR))]

print("Filas originales:", len(df))
print("Filas después del filtro IQR:", len(df_filtrado))

Filas originales: 3072059
Filas después del filtro IQR: 3072059


In [196]:
# Método de la Desviación Absoluta Mediana (MAD) (no se detectan outliers)
median = df[col_num].median()
mad = np.median(np.abs(df[col_num] - median))

# Verificar si MAD es cero para evitar división por cero
if mad == 0:
    print("MAD es 0; revisar la variabilidad de la columna")
else:
    z_score = 0.6745 * (df[col_num] - median) / mad
    
    # Filtrar las filas que no son outliers
    df_filtrado = df[(np.abs(z_score) <= 3.5)]
    
    print("Filas originales:", len(df))
    print("Filas después del filtro MAD:", len(df_filtrado))

Filas originales: 3072059
Filas después del filtro MAD: 3072059


#### Distancia de Mahalanobis

La distancia de Mahalanobis es una medida que permite calcular la distancia entre un punto y una distribución, teniendo en cuenta las correlaciones entre variables en un conjunto de datos multivariado. A diferencia de la distancia euclidiana, que trata a todas las variables por igual, la distancia de Mahalanobis ajusta las distancias según la varianza y covarianza de los datos, lo que la hace especialmente útil para detectar outliers en datos con múltiples variables interrelacionadas.

In [172]:
# Método con la distancia de Mahalanobis (no se encuentran outliers)

data = df[col_num].values

# Calcular la media y la matriz de covarianza
mean = np.mean(data, axis=0)
cov_matrix = np.cov(data, rowvar=False)
inv_cov_matrix = np.linalg.inv(cov_matrix)

# Calcular la distancia de Mahalanobis para cada punto
mahalanobis_distances = [mahalanobis(row, mean, inv_cov_matrix) for row in data]

# Determinar un umbral para identificar outliers
threshold = np.percentile(mahalanobis_distances, 97.5)  # Por ejemplo, el percentil 97.5

# Identificar los outliers
outliers = df[mahalanobis_distances > threshold]

print("Outliers encontrados: ", outliers)

LinAlgError: Singular matrix

### NORMALIZACIÓN/ESTANDARIZACIÓN DE LOS DATOS 

In [197]:
# Estandarización (media = 0 desviación = 1) 

scaler = StandardScaler() 
df[col_num] = scaler.fit_transform(df[col_num]) 

### CODIFICACIÓN DE VARIABLES CATEGÓRICAS 

In [198]:
# Identificar primero variables categóricas 

print(df.dtypes) 

for col in df.select_dtypes(include=['object']).columns: 
    		print(f"{col}: {df[col].nunique()} valores únicos") 

Unnamed: 0                 float64
 Destination Port          float64
 Flow Duration             float64
 Total Fwd Packets         float64
 Total Backward Packets    float64
                            ...   
Idle Mean                  float64
 Idle Std                  float64
 Idle Max                  float64
 Idle Min                  float64
 Label                      object
Length: 80, dtype: object
 Label: 14 valores únicos


In [199]:
# Codificación ordinal para categrorías con orden con el uso de LabelEncoder de la librería de Sci-kit learn
for i in col_cat:
    le = LabelEncoder()
    df[i] = le.fit_transform(df[i]) 

In [201]:
# One-Hot Enccoding para categorías sin orden 

df = pd.get_dummies(df, columns=col_cat, drop_first=True, dummy_na=True)

### GUARDAR DATOS LIMPIOS EN CSV 

In [202]:
df.shape

(3072059, 93)

In [203]:
df.to_csv('../data/datos_limpios.csv')